Shannon Hall

7/29/25

The (dephased) defect of a Hadamard matrix $H$ is the dimension of the tangent space of (dephased) Hadamard matrices at $H$, and it can be a useful tool when studying families of Hadamard matrices. In particular, it gives an upper bound on the number of parameters that a family passing through $H$ can contain. This program implements two algorithms that can be used to find the defect of a given Hadamard matrix. The first method relies on using commutators to find the dimension of the commutator subspace, and the second method uses a formula coming from group theory (with the second method only working for the fourier matrix $F_n$; the formula can be found [here](https://web.math.utk.edu/~rnicoara/defect.pdf)). We include some examples of Hadamard matrices and their defects.

# Defect Function (rank version)

In [5]:
import numpy as np

# This function computes the defect of the given matrix U. We create bases for P=D and Q=UDU*, then calculate all commutators [p, q] from these bases.
# We then use the commutators as the columns of a matrix, and we calculate the defect using the rank of this matrix.
def defect(U):
    n = len(U)
    P_basis = [np.array([[1 if (j == i and k == i) else 0 for k in range(n)] for j in range(n)]) for i in range(n)]
    Q_basis = [U @ p @ U.conj().T for p in P_basis]
    commutators = [p @ q - q @ p for p in P_basis for q in Q_basis]
    commutator_matrix = np.array([sum(commutator.tolist(), []) for commutator in commutators]).T
    return (n - 1)**2 - np.linalg.matrix_rank(commutator_matrix)

## Fourier Matrices

In [12]:
# Get the nth Fourier matrix
def fourier(n):
    x = np.exp(2 * np.pi * 1j / n)
    return np.array([[x**(j * k) for k in range(n)] for j in range(n)])

# Calculate the defect for some Fourier matrices
for n in range(2, 31):
    F = fourier(n)
    print(f'defect of F{n:02} is {defect(F):02}')

defect of F02 is 00
defect of F03 is 00
defect of F04 is 01
defect of F05 is 00
defect of F06 is 04
defect of F07 is 00
defect of F08 is 05
defect of F09 is 04
defect of F10 is 08
defect of F11 is 00
defect of F12 is 17
defect of F13 is 00
defect of F14 is 12
defect of F15 is 16
defect of F16 is 17
defect of F17 is 00
defect of F18 is 28
defect of F19 is 00
defect of F20 is 33
defect of F21 is 24
defect of F22 is 20
defect of F23 is 00
defect of F24 is 53
defect of F25 is 16
defect of F26 is 24
defect of F27 is 28
defect of F28 is 49
defect of F29 is 00
defect of F30 is 76


## Petrescu's Matrix

In [11]:
# Create Petrescu's 7x7 matrix
x = np.exp(2 * np.pi  * 1j / 6)
P_exponents = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 4, 5, 3, 3, 1],
    [0, 4, 1, 3, 5, 3, 1],
    [0, 5, 3, 1, 4, 1, 3],
    [0, 3, 5, 4, 1, 1, 3],
    [0, 3, 3, 1, 1, 4, 5],
    [0, 1, 1, 3, 3, 5, 4]])
P = np.power(x, P_exponents)

# Calculate the defect of Petrescu's matrix
print(f'defect of P7 is {defect(P)}')

defect of P7 is 3


## Petrescu's Family

### Generic Values

In [ ]:
# Create a matrix from Petrescu's family of 7x7 matrices
def petrescu(c):
    Pc = np.copy(P)
    c_conjugate = np.conjugate(c)
    for i in range(1, 3):
        for j in range(1, 3):
            Pc[i, j] *= c
            Pc[i + 2, j + 2] *= c_conjugate
    return Pc

# Calculate the defect for some generic matrices in Petrescu's family
generic_values = np.linspace(0, 2 * np.pi, num=10, endpoint=False)
for t in generic_values:
    c = np.exp(1j * t)
    Pc = petrescu(c)
    print(f'defect of P7(exp(i * {t:.2f})) is {defect(Pc)}')

defect of P7(exp(i * 0.00)) is 3
defect of P7(exp(i * 0.63)) is 2
defect of P7(exp(i * 1.26)) is 2
defect of P7(exp(i * 1.88)) is 2
defect of P7(exp(i * 2.51)) is 2
defect of P7(exp(i * 3.14)) is 3
defect of P7(exp(i * 3.77)) is 2
defect of P7(exp(i * 4.40)) is 2
defect of P7(exp(i * 5.03)) is 2
defect of P7(exp(i * 5.65)) is 2


### Specific Values

In [23]:
# Calculate the defect for some specific matrices in Petrescu's family
specific_values = [0, np.pi / 3, 2 * np.pi / 3, np.pi]
for t in specific_values:
    c = np.exp(1j * t)
    Pc = petrescu(c)
    print(f'defect of P7(exp(i * {t:.2f})) is {defect(Pc)}')

defect of P7(exp(i * 0.00)) is 3
defect of P7(exp(i * 1.05)) is 3
defect of P7(exp(i * 2.09)) is 3
defect of P7(exp(i * 3.14)) is 3


# Defect Function (group version)

In [24]:
# This function calculates the (undephased) defect of the group Zn, or, equivalently, the defect of the Fourier matrix Fn
# This is done by using the order of each element g in the group Zn
ord = lambda g, n : n / np.gcd(g, n)
undephased_group_defect1 = lambda n : sum(n / ord(g, n) for g in range(n)) - 2 * n + 1

# After simplifying the above formula, we get simpler expressions for the defect of the group Zn
undephased_group_defect2 = lambda n : sum(np.gcd(g, n) for g in range(n))
dephased_group_defect = lambda n : sum(np.gcd(g, n) for g in range(n)) - 2 * n + 1

# Calculate the defect for the group Zn
for n in range(2, 31):
    print(f'defect of Z{n:02} is {dephased_group_defect(n):02}')

defect of Z02 is 00
defect of Z03 is 00
defect of Z04 is 01
defect of Z05 is 00
defect of Z06 is 04
defect of Z07 is 00
defect of Z08 is 05
defect of Z09 is 04
defect of Z10 is 08
defect of Z11 is 00
defect of Z12 is 17
defect of Z13 is 00
defect of Z14 is 12
defect of Z15 is 16
defect of Z16 is 17
defect of Z17 is 00
defect of Z18 is 28
defect of Z19 is 00
defect of Z20 is 33
defect of Z21 is 24
defect of Z22 is 20
defect of Z23 is 00
defect of Z24 is 53
defect of Z25 is 16
defect of Z26 is 24
defect of Z27 is 28
defect of Z28 is 49
defect of Z29 is 00
defect of Z30 is 76
